- notebook : https://github.com/donarts/sourcecode/blob/main/pytorch/06_bert
- train : https://swlock.blogspot.com/2023/03/bert-nsmc-pytorch-train.html
- predict : https://swlock.blogspot.com/2023/03/bert-nsmc-pytorch-predict.html

In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from transformers import AutoTokenizer

In [2]:
import pandas as pd

In [3]:
import torch

In [4]:
import numpy as np

In [ ]:
train 과 같은 pretrain된 토크나이저를 준비합니다.

In [5]:
pretrained_model_name="beomi/kcbert-base"
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name
)

예제 데이터를 준비하였습니다. 문구는 nsmc 에 없는 내용으로 제가 임의로 만든 문장입니다.

In [6]:
test_data = pd.DataFrame({"texts":["이러한 내용을 지금까지 몰랐다니, 정말 다시 보고 싶다.",
                                   "오 좋은데","이상하다",
                                   "ㅋㅋ나는 재미있는지 잘모르겠다",
                                   "오랜만에 굉장히 훌륭한 만남"]})

In [7]:
test_data

,texts
0,"이러한 내용을 지금까지 몰랐다니, 정말 다시 보고 싶다."
1,오 좋은데
2,이상하다
3,ㅋㅋ나는 재미있는지 잘모르겠다
4,오랜만에 굉장히 훌륭한 만남


In [8]:
tokenized_test_sentences = tokenizer(
    list(test_data.texts),
    return_tensors="pt",
    padding=True,
    truncation=True,
)

In [9]:
tokenized_test_sentences

{'input_ids': tensor([[    2, 16702, 15443,  9461, 14802,  4198,    15,  8050,  8245,  8198,
          8975,    17,     3],
        [    2,  2298, 11827,     3,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [    2, 16644,     3,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0],
        [    2,  7970,  8862, 10827, 14847,  2483, 11651,  8077,     3,     0,
             0,     0,     0],
        [    2, 20646, 17611, 13659, 29869,     3,     0,     0,     0,     0,
             0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    

이 부분이 중요한데, predict는 label이 필요하지 않습니다, 그래서 데이터 로딩하는 부분은 train 부분에서 label 부분을 삭제하고 새로 만들었습니다.

In [10]:
class DataloaderDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [11]:
pred_dataset = DataloaderDataset(tokenized_test_sentences)

In [12]:
len(pred_dataset)

5

이전에 finetuning 된 데이터를 load 해줍니다. 이렇게 사용하기위해서 train 마지막에 저장을 했습니다.

In [13]:
model_loaded = AutoModelForSequenceClassification.from_pretrained("trained_model")

In [14]:
trainer = Trainer(model = model_loaded)

아래 코드가 trainer로 predict 하는 코드 입니다.

In [15]:
pred_results = trainer.predict(pred_dataset)

***** Running Prediction *****
  Num examples = 5
  Batch size = 8
C:\Users\jun\AppData\Local\Temp\ipykernel_10724\3071759006.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [16]:
print(pred_results)

PredictionOutput(predictions=array([[-1.9505684 ,  1.5610104 ],
       [-0.6223819 ,  0.6025323 ],
       [ 1.6676984 , -2.0425177 ],
       [ 0.28711993, -0.69983894],
       [-2.316752  ,  1.851989  ]], dtype=float32), label_ids=None, metrics={'test_runtime': 7.1129, 'test_samples_per_second': 0.703, 'test_steps_per_second': 0.141})


In [17]:
pred_results.predictions

array([[-1.9505684 ,  1.5610104 ],
       [-0.6223819 ,  0.6025323 ],
       [ 1.6676984 , -2.0425177 ],
       [ 0.28711993, -0.69983894],
       [-2.316752  ,  1.851989  ]], dtype=float32)

np.argmax로 예측된 값중 가장 큰값을 가져오도록 합니다.

In [18]:
predictions = np.argmax(pred_results.predictions, axis=-1)

In [20]:
test_data["labels"]=predictions

원본 데이터와 예측한 값을 같이 표기해봤습니다. 0은 부정의미이고 1은 긍정 의미입니다.

In [21]:
test_data

,texts,labels
0,"이러한 내용을 지금까지 몰랐다니, 정말 다시 보고 싶다.",1
1,오 좋은데,1
2,이상하다,0
3,ㅋㅋ나는 재미있는지 잘모르겠다,0
4,오랜만에 굉장히 훌륭한 만남,1
